<a href="https://colab.research.google.com/github/kimhalyn/OPEN-API/blob/main/walcode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 라이브러리
# colab에 xmltodict 라이브러리가 기본 설치되지 않으므로 별도 설치(외부 라이브러리)
!pip install xmltodict 
from urllib.request import urlopen, Request
from urllib.parse import urlencode, unquote, quote_plus
from bs4 import BeautifulSoup
import pandas as pd
import requests, xmltodict, json
from xml.etree.ElementTree import Element, SubElement, ElementTree
import xml.etree.ElementTree as ET
import pickle
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:

import requests, bs4
import pandas as pd
from lxml import html
from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote_plus, unquote
from time import sleep
import csv

url = 'http://opendata.kwater.or.kr/openapi-data/service/pubd/dam/excllncobsrvt/wal/list' #!!확인
key = 'tMF2MGGUmsNNPf15Nr91+UA4YVQiWXVjLS7R0odxGSR0yfNjcHFkdRpCT6hXbILlBP1T+TnDJYz1kkDjiaIiQg==' #!!확인
 
api_key = requests.utils.unquote(key) #디코딩

# 댐코드 불러오기
data = pd.read_csv('/content/drive/MyDrive/water_resource_csv/damCode.csv') #!!확인

# 필요한 코드 조회
damcode = data.iloc[:, 1] #!!열 확인
xmlrows = []
for code in damcode:

  Params = '?' + urlencode({quote_plus('ServiceKey') : api_key,
                                quote_plus('damcode') : code})  
#  print(Params)

  res = requests.get(url + Params).text.encode('utf-8') # 요청에 대한 응답내용(.encode빼기도 함)
  sleep(0.5)
  print(res) 
  
  xmlobj = bs4.BeautifulSoup(res, 'lxml') #lxml 파서(구문분석)  'lxml-xml'
  print(xmlobj)
  
  #item 태그 분리
  rows = xmlobj.findAll('item') # 매치되는 문자열을 리스트로 리턴
  xmlrows.append(rows)
  
print (xmlrows)

b'<?xml version="1.0" encoding="UTF-8" standalone="yes"?><response><header><resultCode>00</resultCode><resultMsg>NORMAL SERVICE.</resultMsg></header><body><items/></body></response>'
<?xml version="1.0" encoding="UTF-8" standalone="yes"?><html><body><response><header><resultcode>00</resultcode><resultmsg>NORMAL SERVICE.</resultmsg></header><items></items></response></body></html>
b'<?xml version="1.0" encoding="UTF-8" standalone="yes"?><response><header><resultCode>00</resultCode><resultMsg>NORMAL SERVICE.</resultMsg></header><body><items><item><obsrvtNm>\xea\xb0\x95\xec\xa0\x95\xea\xb3\xa0\xeb\xa0\xb9\xeb\xb3\xb4\xec\x83\x81\xeb\xa5\x98</obsrvtNm><walobsrvtcode>2011602</walobsrvtcode></item><item><obsrvtNm>\xea\xb0\x95\xec\xa0\x95\xea\xb3\xa0\xeb\xa0\xb9\xeb\xb3\xb4\xed\x95\x98\xeb\xa5\x98</obsrvtNm><walobsrvtcode>2011655</walobsrvtcode></item></items></body></response>'
<?xml version="1.0" encoding="UTF-8" standalone="yes"?><html><body><response><header><resultcode>00</resultcode><re

In [15]:
print(xmlrows)

[[], [<item><obsrvtnm>강정고령보상류</obsrvtnm><walobsrvtcode>2011602</walobsrvtcode></item>, <item><obsrvtnm>강정고령보하류</obsrvtnm><walobsrvtcode>2011655</walobsrvtcode></item>], [], [<item><obsrvtnm>공주보상류</obsrvtnm><walobsrvtcode>3012601</walobsrvtcode></item>, <item><obsrvtnm>공주보하류</obsrvtnm><walobsrvtcode>3012621</walobsrvtcode></item>], [<item><obsrvtnm>삼척시(갈밭교)</obsrvtnm><walobsrvtcode>9000266</walobsrvtcode></item>, <item><obsrvtnm>삼척시(광동교)</obsrvtnm><walobsrvtcode>9000264</walobsrvtcode></item>, <item><obsrvtnm>삼척시(번천교)</obsrvtnm><walobsrvtcode>9000268</walobsrvtcode></item>, <item><obsrvtnm>태백시(무사교)</obsrvtnm><walobsrvtcode>9000265</walobsrvtcode></item>], [], [<item><obsrvtnm>거제시(구천교)</obsrvtnm><walobsrvtcode>9000256</walobsrvtcode></item>], [], [], [], [<item><obsrvtnm>구포(부산)</obsrvtnm><walobsrvtcode>8000099</walobsrvtcode></item>, <item><obsrvtnm>낙동강하굿둑우안상류(부산)</obsrvtnm><walobsrvtcode>2022706</walobsrvtcode></item>, <item><obsrvtnm>낙동강하굿둑우안하류(부산)</obsrvtnm><walobsrvtcode>2022710</wal

In [ ]:

filePath = '/content/drive/MyDrive/water_resource_csv/xmlrows.xml'


with open(filePath, 'wb') as lf:
    pickle.dump(xmlrows, lf)


with open(filePath, 'rb') as lf:
    readList = pickle.load(lf)


In [13]:
rowList = []
nameList = []
columnList = []
xmlrowsLen = len(xmlrows)
target = 0
for i in range(0, xmlrowsLen):
    
   
    if len(xmlrows[i]) == 0:
       target = target + 1
      
    else:
        break #반복문 종료
columns = xmlrows[target][0].find_all()
columnsLen = len(columns)
for i in range(0, columnsLen):
    nameList.append(columns[i].name)
nameList = ["Damcode"] + nameList

In [17]:
xmlrows[0]

[]

In [18]:
total = []
damcodeLen = len(damcode)


def el_code(a): 
    columnList = []
    eachrows = xmlrows[a]
    eachrowslen = len(eachrows)
    for i in range(0, eachrowslen):
        columns = eachrows[i].find_all()
        columnsLen = len(columns)
        for j in range(0, columnsLen):
            eachColumn = columns[j].text 
            columnList.append(eachColumn)
        columnList = [damcode[a]] + columnList
        rowList.append(columnList)
        columnList = []
    return rowList



for a in range(0, damcodeLen):
    rowList=el_code(a)
    total.append(rowList)

result = pd.DataFrame(total[0], columns=nameList)

In [19]:
result.to_csv("/content/drive/MyDrive/water_resource_csv/walcode_1.csv",encoding="utf-8-sig")

In [ ]:
result